In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import PorterStemmer
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from wordcloud import WordCloud
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
import pickle
import re
import os

from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle
import os

from selenium import webdriver
from bs4 import BeautifulSoup
import time 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kushajra\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
driver = webdriver.Chrome()

In [4]:
#You will need to enter the amazon reviews website link for the product below
driver.get('https://www.amazon.in/Dell-Vostro-Metal-i5-1235U-35-56Cms/product-reviews/B0BQJ7WR2R/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')

html_data = BeautifulSoup(driver.page_source, 'html.parser')

# Initial URL of the first page of reviews
url = 'https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'+'=1'
# Initialize lists to store the review data
names = []
ratings = []
rating_dates = []
titles = []
reviews_text = []

while url is not None:
    driver.get(url)
    time.sleep(5)  # Wait for the page to load
    html_data = BeautifulSoup(driver.page_source, 'html.parser')
    reviews = html_data.find_all('div', {'data-hook': 'review'})
    
    for review in reviews:
        name = review.find('span', {'class': 'a-profile-name'}).text
        names.append(name.strip())
        rating = review.find('span', {'class': 'a-icon-alt'}).text
        ratings.append(rating)
        rating_date = review.find('span', {'data-hook': 'review-date'}).text
        rating_dates.append(rating_date)
        title = review.find('a', {'data-hook': 'review-title'}).text
        titles.append(title.strip())
        review_text = review.find('span', {'data-hook': 'review-body'}).text
        reviews_text.append(review_text.strip())
    
    # Check for the next page link
    url_check = html_data.find('li', {'class': 'a-last'})
    if url_check is None or 'a-disabled' in url_check['class']:
        url = None
    else:
        url = 'https://www.amazon.in' + url_check.a['href']
    
    print(url)

https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber=2&reviewerType=all_reviews%3D1
https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_arp_d_paging_btm_3?ie=UTF8&pageNumber=3&reviewerType=all_reviews%3D1
https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_arp_d_paging_btm_4?ie=UTF8&pageNumber=4&reviewerType=all_reviews%3D1
https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_arp_d_paging_btm_5?ie=UTF8&pageNumber=5&reviewerType=all_reviews%3D1
https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_arp_d_paging_btm_6?ie=UTF8&pageNumber=6&reviewerType=all_reviews%3D1
https://www.amazon.in/Dyazo-Upgraded-Adjustable-Notebooks-26/product-reviews/B09X67KLXH/ref=cm_cr_arp_d_paging_btm_7?ie=UTF8&pageNumber=7&reviewerType=all_reviews%3D

In [5]:
df = pd.DataFrame({'profile_name':names,
                    'rating':ratings,
                     'rating_date':rating_dates,
                     'title':titles,
                     'review_text':reviews_text
                     })
df

,profile_name,rating,rating_date,title,review_text
0,Praveen Kumar,5.0 out of 5 stars,Reviewed in India on 6 May 2024,5.0 out of 5 stars\nNice Product,"Light weight, runs smoothly and quietly, value..."
1,S.P,4.0 out of 5 stars,Reviewed in India on 15 April 2024,4.0 out of 5 stars\nValue for money,This laptop cooling fan is a game-changer! It ...
2,Seattle Rodrigues,5.0 out of 5 stars,Reviewed in India on 23 May 2024,5.0 out of 5 stars\nGood cooling capacity,"Looks good,Good adjustment lengths however nee..."
3,Nitin Shrivas,4.0 out of 5 stars,Reviewed in India on 21 May 2024,4.0 out of 5 stars\nNice product,It is really a good cooling pad and also very ...
4,Amazon Customer,5.0 out of 5 stars,Reviewed in India on 5 May 2024,5.0 out of 5 stars\nWorks as advertised and ru...,Noiseless and smooth running. Using it and wor...
5,Ritesh Patel,3.0 out of 5 stars,Reviewed in India on 15 May 2024,3.0 out of 5 stars\nServrs the purpose,Product is good and build quality also good.Bu...
6,Manojkumar,5.0 out of 5 stars,Reviewed in India on 6 May 2024,5.0 out of 5 stars\nGood,The media could not be loaded.\n ...
7,Aditya,4.0 out of 5 stars,Reviewed in India on 24 April 2024,4.0 out of 5 stars\nGood For Price but can be ...,Everything is good at this price range but bui...
8,Harshit J.,4.0 out of 5 stars,Reviewed in India on 20 May 2024,"4.0 out of 5 stars\nNice product, Can consider...",I liked the build quality offered in this pric...
9,Amazon Customer,4.0 out of 5 stars,Reviewed in India on 25 May 2024,4.0 out of 5 stars\nHp,Does the Job.. My HP laptop this fan speed is ...


In [8]:
df.to_csv('Dell15.csv')

In [1]:
#This is so kush can see that the code works (kys kush (jk (not really)))
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pickle
import os

#This is just to load the data

data = pd.read_csv(r"amazon_alexa.tsv", delimiter = '\t', quoting = 3)
print(f"Dataset shape : {data.shape}")
data[data['verified_reviews'].isna() == True]
data.dropna(inplace=True)

# Load the data
data = pd.read_csv(r"amazon_alexa.tsv", delimiter='\t', quoting=3)

# Preprocess the reviews
corpus = []
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

for i in range(data.shape[0]):
    review = data.iloc[i]['verified_reviews']
    if not isinstance(review, str):
        review = ''
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower().split()
    review = [stemmer.stem(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

# Create the bag of words model
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus).toarray()
y = data['feedback'].values

# Save the CountVectorizer
os.makedirs('Models', exist_ok=True)
with open('Models/countVectorizer.pkl', 'wb') as file:
    pickle.dump(cv, file)

# Split the data into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=15)

# Scale the features
scaler = MinMaxScaler()
X_train_scl = scaler.fit_transform(X_train)
X_test_scl = scaler.transform(X_test)

# Save the scaler
pickle.dump(scaler, open('Models/scaler.pkl', 'wb'))

# Train the XGBoost model
model_xgb = XGBClassifier()
model_xgb.fit(X_train_scl, y_train)

# Predict sentiment for the test set
y_preds = model_xgb.predict(X_test)


# Save the XGBoost classifier
pickle.dump(model_xgb, open('Models/model_xgb.pkl', 'wb'))

# Sentiment analysis on the first 10 reviews
first_10_reviews = data['verified_reviews'][:10]

# Preprocess the first 10 reviews
first_10_reviews_processed = [re.sub('[^a-zA-Z]', ' ', review).lower() if isinstance(review, str) else '' for review in first_10_reviews]
first_10_reviews_processed = [' '.join([stemmer.stem(word) for word in review.split() if word not in stop_words]) for review in first_10_reviews_processed]

# Transform and scale the first 10 reviews
first_10_reviews_transformed = cv.transform(first_10_reviews_processed)
first_10_reviews_scl = scaler.transform(first_10_reviews_transformed.toarray())
first_10_predictions = model_xgb.predict(first_10_reviews_scl)

# Print sentiment predictions for the first 10 reviews
for i in range(10):
    print(f"Review: {first_10_reviews.iloc[i]}")
    print(f"Sentiment: {'Positive' if first_10_predictions[i] == 1 else 'Negative'}")
    print()

# Create a table of reviews categorized by star ratings
review_table = data[['rating', 'verified_reviews']]
review_table_grouped = review_table.groupby('rating')['verified_reviews'].apply(list).reset_index()

# Display the table
print("Reviews categorized by star ratings:")
print(review_table_grouped)

# Save the review table as a CSV
review_table_grouped.to_csv('review_table_grouped.csv', index=False)

Dataset shape : (3150, 5)
Review: Love my Echo!
Sentiment: Positive

Review: Loved it!
Sentiment: Positive

Review: "Sometimes while playing a game, you can answer a question correctly but Alexa says you got it wrong and answers the same as you.  I like being able to turn lights on and off while away from home."
Sentiment: Positive

Review: "I have had a lot of fun with this thing. My 4 yr old learns about dinosaurs, i control the lights and play games like categories. Has nice sound when playing music as well."
Sentiment: Positive

Review: Music
Sentiment: Positive

Review: "I received the echo as a gift. I needed another Bluetooth or something to play music easily accessible, and found this smart speaker. Can’t wait to see what else it can do."
Sentiment: Positive

Review: "Without having a cellphone, I cannot use many of her features. I have an iPad but do not see that of any use.  It IS a great alarm.  If u r almost deaf, you can hear her alarm in the bedroom from out in the living

In [12]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import pickle
import os

# Load the pre-trained components
with open('Models/countVectorizer.pkl', 'rb') as file:
    cv = pickle.load(file)

with open('Models/scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('Models/model_xgb.pkl', 'rb') as file:
    model_xgb = pickle.load(file)

# Load the new data
dell15_data = pd.read_csv(r"dell15.csv")

# Preprocess the new reviews
corpus = []
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

for i in range(dell15_data.shape[0]):
    review = dell15_data.iloc[i]['review_text']
    if not isinstance(review, str):
        review = ''
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower().split()
    review = [stemmer.stem(word) for word in review if word not in stop_words]
    review = ' '.join(review)
    corpus.append(review)

# Transform and scale the new reviews
X_new = cv.transform(corpus).toarray()
X_new_scl = scaler.transform(X_new)

# Predict sentiment probabilities for the new reviews
new_predictions_proba = model_xgb.predict_proba(X_new_scl)[:, 1]  # Get the probability for class 1 (positive)

# Scale probabilities to range from -10 to 10
sentiment_scores = new_predictions_proba * 20 - 10

# Add predictions to the dataframe
dell15_data['Sentiment_Score'] = sentiment_scores

# Reorder columns to have 'Sentiment_Score' right after 'review_text'
cols = list(dell15_data.columns)
review_text_index = cols.index('review_text')
cols.insert(review_text_index + 1, cols.pop(cols.index('Sentiment_Score')))
dell15_data = dell15_data[cols]

# Save the results to a new CSV in a specific directory
dell15_data.to_csv('C:/Users/kushajra/Documents/dell15_reviews_with_sentiment_score.csv', index=False)

# Display the dataframe with the new Sentiment_Score column
dell15_data  # Display first 10 rows for verification

,Unnamed: 0,profile_name,rating,rating_date,title,review_text,Sentiment_Score,Sentiment
0,0,Praveen Kumar,5.0 out of 5 stars,Reviewed in India on 6 May 2024,5.0 out of 5 stars\nNice Product,"Light weight, runs smoothly and quietly, value...",4.694082,Positive
1,1,S.P,4.0 out of 5 stars,Reviewed in India on 15 April 2024,4.0 out of 5 stars\nValue for money,This laptop cooling fan is a game-changer! It ...,9.939739,Positive
2,2,Seattle Rodrigues,5.0 out of 5 stars,Reviewed in India on 23 May 2024,5.0 out of 5 stars\nGood cooling capacity,"Looks good,Good adjustment lengths however nee...",9.951702,Positive
3,3,Nitin Shrivas,4.0 out of 5 stars,Reviewed in India on 21 May 2024,4.0 out of 5 stars\nNice product,It is really a good cooling pad and also very ...,8.079596,Positive
4,4,Amazon Customer,5.0 out of 5 stars,Reviewed in India on 5 May 2024,5.0 out of 5 stars\nWorks as advertised and ru...,Noiseless and smooth running. Using it and wor...,8.267307,Positive
...,...,...,...,...,...,...,...,...
95,95,AMITAVA ROY,5.0 out of 5 stars,Reviewed in India on 26 April 2024,5.0 out of 5 stars\nValue for money,"Attractive look, easy to use and nice working.",9.937876,Positive
96,96,Devarsh Bhatt,2.0 out of 5 stars,Reviewed in India on 3 May 2024,"2.0 out of 5 stars\nNot bad, not good","Look wise it is amazing product, but does not ...",8.637037,Positive
97,97,Ashish Walia,5.0 out of 5 stars,Reviewed in India on 23 April 2024,5.0 out of 5 stars\nNice,Money wroth,5.470534,Positive
98,98,A.Abdul Salam,5.0 out of 5 stars,Reviewed in India on 22 March 2024,5.0 out of 5 stars\nDyazo laptop cooling Produ...,Dyazo laptop cooling pad is worth buying it. T...,9.773415,Positive
